<a href="https://colab.research.google.com/github/RobertBWeidlich/Colab_Notebooks/blob/main/Budget_2025_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
# 1. manually export data from XLS to CSV file:
# (in Excel)
#   Excel -> Open sheet to be analyzed -> File -> Export ->
#       Change File Type -> CSV


# move csv data from local Windows host to local linux server
# for analysis

from google.colab import files
uploaded = files.upload()
print(uploaded)
file_name = next(iter(uploaded))
#data = uploaded[file_name]
#print(data)

Saving Budget-2024-2025-06-27-1340.csv to Budget-2024-2025-06-27-1340.csv
{'Budget-2024-2025-06-27-1340.csv': b'Timestamp,Description,Check #,Category,Source Account,Priority,Amount,Comments,,,,,,,,,\r\n,,,,,,,,,,,,,,,,\r\n2024-01-01-2399,DigitalOcean.com,,computer,amex-13003,3,6.00,,,,,,,,,,\r\n2024-01-01-2399,nytimes,,newspapers,amex-13003,3,57.19,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,\r\n2024-01-02-2399,Target,,house,debit,5,28.34,,,,,,,,,,\r\n2024-01-02-2399,Total Wine,,groceries-bob,debit,3,77.86,,,,,,,,,,\r\n2024-01-02-2399,Office Depot,,house,debit,5,42.90,,,,,,,,,,\r\n2024-01-02-2399,Arbys,,restaurants,debit,3,17.03,,,,,,,,,,\r\n2024-01-02-2399,AWS - Web Services,,computer,amex-13003,3,1.20,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,\r\n2024-01-03-2399,Giant,,pharmacy,debit,5,35.00,,,,,,,,,,\r\n2024-01-03-2399,Giant,,groceries-bob,debit,5,60.00,,,,,,,,,,\r\n2024-01-03-2399,Prime Video,,entertainment,amex-13003,5,19.99,,,,,,,,,,\r\n2024-01-03-2399,Paypal RestonRadio,,unknown,amex-13003,3,19.99,,,,,,,,

In [ ]:
"""
type(uploaded)
import pprint as pp
#pp.pprint(uploaded)
kk = uploaded.keys()
print(type(kk))
print(kk)
file_name = next(iter(uploaded))
print(file_name)
"""

In [2]:
# 2. load data into pandas and print it out.
import pandas as pd

dtype_spec = {
    "Description": str,
    "Category": str,
    "Source Account": str,
    "Check #": "Int64",     # Use nullable integer type
    "Priority": "Int64",
    "Amount": float
}

df = pd.read_csv(file_name,
                 encoding="cp1252",
                 usecols=range(10))

#df = pd.read_csv(file_name, dtype=dtype_spec, parse_dates=["Timestamp"])

print(df.dtypes)

# drop all rows where all cells are NaN
df = df.dropna(how='all')

# set "Check #" column to integer
# openai - doesn't work
#df["Check #"] = pd.to_numeric(df["Check #"], errors="coerce").astype("Int64")
# gemini - doesn't work
#df["Check #"] = pd.to_numeric(df["Check #"], errors="coerce").astype("Int64")

# this does not work with colab
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#  print(df)
# this DOES work with colab
#print(df.to_string())
print(df.head(20).to_string())


Timestamp          object
Description        object
Check #           float64
Category           object
Source Account     object
Priority          float64
Amount             object
Comments           object
Unnamed: 8        float64
Unnamed: 9        float64
dtype: object
          Timestamp         Description  Check #          Category Source Account  Priority  Amount Comments  Unnamed: 8  Unnamed: 9
1   2024-01-01-2399    DigitalOcean.com      NaN          computer     amex-13003       3.0    6.00      NaN         NaN         NaN
2   2024-01-01-2399             nytimes      NaN        newspapers     amex-13003       3.0   57.19      NaN         NaN         NaN
4   2024-01-02-2399              Target      NaN             house          debit       5.0   28.34      NaN         NaN         NaN
5   2024-01-02-2399          Total Wine      NaN     groceries-bob          debit       3.0   77.86      NaN         NaN         NaN
6   2024-01-02-2399        Office Depot      NaN             

In [ ]:
# 3. sort by the "Category" column to check for variations category names
df_sorted = df.sort_values(by="Category", ascending=True)
#print(df_sorted.head(20).to_string())
print(df_sorted.head(5000).to_string())

In [ ]:
# 4. now sort by the "Source Account" column to check for variations
df_sorted = df.sort_values(by="Source Account", ascending=True)
#print(df_sorted.head(20).to_string())
print(df_sorted.head(5000).to_string())

In [3]:

# take a list of numbers in various formats ("1", "1.00", "$1.00")
# and return sum in currency format
# NOTE: don't use this; instead use:
#   [68.93, 12.50, 100.25]
#
def sum_of_list_currency_format(cur_list):
  sum = 0.0
  for cur in cur_list:
    #print(type(cur))
    if type(cur) == str:
      cur = cur.replace('$', '').replace(',', '')
    sum += float(cur)
  return(round(sum, 2))

test_cur_list = ["13.00", "$54.98", 1]
sum_of_list_currency_format(test_cur_list)

68.98

In [4]:
# 5. Analysis: for each "Category" column, find sum of "Amount" column
#
# 5A. set some columns to lower case
#print(type(df['Category'].astype(str).str.lower()))
df['Category'] = df['Category'].astype(str).str.lower()

# 5B. get list of unique categories and unique major-categories.
#     for a category "abc-345", "abc" is the major category.
#
unique_categories = sorted(df['Category'].unique().tolist())
print("unique_categories:")
print(unique_categories)
print("unique_major_categories:")
unique_major_categories = sorted(set(item.split('-')[0] for item in unique_categories))
print(unique_major_categories)

unique_categories:
['auto-gas', 'auto-maintenance', 'charity', 'computer', 'entertainment', 'groceries', 'groceries-bob', 'groceries-margie', 'house', 'insurance-auto', 'insurance-house', 'insurance-life', 'kate', 'lawngarden', 'loans', 'medical', 'medical-margie', 'misc', 'misc-bob', 'misc-margie', 'newspapers', 'pharmacy', 'restaurants', 'restaurants-bob', 'restaurants-margie', 'tax', 'unknown', 'utilities']
unique_major_categories:
['auto', 'charity', 'computer', 'entertainment', 'groceries', 'house', 'insurance', 'kate', 'lawngarden', 'loans', 'medical', 'misc', 'newspapers', 'pharmacy', 'restaurants', 'tax', 'unknown', 'utilities']


In [6]:
# 6. for each category, print all records matching that category, along with the sum:
acc_cat_sums = [] # accumulate data, implemented as list of tuples.
                  # each tuple contains a category and the sum
                  # off all entries in that category
agg_sums = 0.0

for cat in unique_categories:
  print()
  print(f"category: {cat}")
  print("========")
  filtered_df = df[df['Category'].str.lower() == cat]
  print(filtered_df.head(500).to_string())
  # get sum of "Amount" column using sum_of_list_currency_format()
  amount_list = filtered_df["Amount"].tolist()
  #print(amount_list)
  sum = sum_of_list_currency_format(amount_list)
  acc_cat_sums.append((cat, sum))
  agg_sums += sum
  print(f"sum: {sum}")
  print("########")
  print(acc_cat_sums)
  print(agg_sums)
  print("########")
  print("========")



category: auto-gas
            Timestamp          Description  Check #  Category Source Account  Priority Amount Comments  Unnamed: 8  Unnamed: 9
16    2024-01-03-2399                Exxon      NaN  auto-gas        mc-8752       5.0  43.00      NaN         NaN         NaN
52    2024-01-14-2399               Shello      NaN  auto-gas          shell       5.0  39.32      NaN         NaN         NaN
90    2024-01-25-2399                Exxon      NaN  auto-gas          exxon       5.0  24.73      NaN         NaN         NaN
168   2024-02-16-2399                Shell      NaN  auto-gas          shell       5.0  38.17      NaN         NaN         NaN
250   2024-03-12-2399                Shell      NaN  auto-gas          shell       5.0  38.65      NaN         NaN         NaN
325   2024-03-27-2399       Exxon Sterling      NaN  auto-gas        mc-8752       5.0  44.41      NaN         NaN         NaN
350   2025-04-03-1044                Shell      NaN  auto-gas          shell       5.0  42.